In [1]:
import pyaudio
import wave
import sounddevice
import sys
from tqdm import tqdm
from IPython.display import Audio
from pydub import AudioSegment

print("script starts here")
# speaker_name = input("Name: ")
speaker_name = "Silence"
# two recordings in Arabic and two in English. each recording is 60 seconds
recording_number = "1"
# recording_number = input("Recording id (e1, e2, a1, a2): ")
# recording_length = int(input("Length in seocnds: "))
recording_length = 30




CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = int(recording_length) + 5.1
WAVE_OUTPUT_FILENAME = speaker_name + "_" + recording_number +".wav"

file_path = "" + WAVE_OUTPUT_FILENAME

record = True
vad = False

if record:
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Started recording...")
    frames = []

    for i in tqdm(range(0, int(RATE / CHUNK * RECORD_SECONDS))):
        data = stream.read(CHUNK)
        frames.append(data)


    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(file_path, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    print("Done recording! - T")

C:\Users\Sherif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


script starts here
Started recording...


100%|██████████| 548/548 [00:35<00:00, 15.63it/s]

Done recording! - T


# VAD

In [2]:
import torch

output_file = file_path.split('.')[0]+ '.wav'

if vad:
    model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                                model='silero_vad',
                                force_reload=True,
                                onnx=False)

    (get_speech_timestamps,
    save_audio,
    read_audio,
    VADIterator,
    collect_chunks) = utils

    file = WAVE_OUTPUT_FILENAME
    Audio(file)

    output_file = file_path.split('.')[0]+'_only_speech'+ '.wav'
    print(file)
    wav = read_audio(file, sampling_rate=RATE)
    # get speech timestamps from full audio file
    speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=RATE)
    # merge all speech chunks to one audio
    if speech_timestamps:
            save_audio(output_file,
                    collect_chunks(speech_timestamps, wav), sampling_rate=RATE) 
            Audio(output_file)
    else:
            print("No activity detected")
    Audio(output_file)

# Segment

In [3]:
from scipy.io import wavfile
import os

def trim_wav(input_file, output_folder, segment_name, offset=0, length=3):
    # Read the WAV file
    sample_rate, audio_data = wavfile.read(input_file)

    # Define the duration of each segment in samples (3 seconds)
    segment_duration = length * sample_rate

    # Calculate the number of segments
    num_segments = len(audio_data) // segment_duration
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Trim the audio into segments
    for i in range(num_segments):
        start_sample = i * segment_duration
        end_sample = (i + 1) * segment_duration
        segment = audio_data[start_sample:end_sample]
        # Save each segment as a separate WAV file
        output_file = os.path.join(output_folder, f"{segment_name}_{i+1+offset}.wav")
        wavfile.write(output_file, sample_rate, segment)

In [4]:
output_folder = output_file.split('.')[0]

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Directory '{output_folder}' created successfully.")
else:
    print(f"Directory '{output_folder}' already exists.")

trim_wav(output_file, output_folder, output_folder, 0, 2)

Directory 'Silence_1' already exists.


# Normalize

In [5]:
import glob

# Replace 'directory_path' with the actual directory path you want to read files from
directory_path = output_folder

# Use glob to get a list of all WAV files in the directory recursively
wav_files = glob.glob(directory_path + '/**/*.wav', recursive=True)

# Print the list of WAV files
print(wav_files)


['Silence_1\\Silence_1_1.wav', 'Silence_1\\Silence_1_10.wav', 'Silence_1\\Silence_1_11.wav', 'Silence_1\\Silence_1_12.wav', 'Silence_1\\Silence_1_13.wav', 'Silence_1\\Silence_1_14.wav', 'Silence_1\\Silence_1_15.wav', 'Silence_1\\Silence_1_16.wav', 'Silence_1\\Silence_1_17.wav', 'Silence_1\\Silence_1_2.wav', 'Silence_1\\Silence_1_3.wav', 'Silence_1\\Silence_1_4.wav', 'Silence_1\\Silence_1_5.wav', 'Silence_1\\Silence_1_6.wav', 'Silence_1\\Silence_1_7.wav', 'Silence_1\\Silence_1_8.wav', 'Silence_1\\Silence_1_9.wav']


In [6]:
for file_name in wav_files:
    with wave.open(file_name, 'rb') as wav_file:
        # Get the audio file properties
        sample_width = wav_file.getsampwidth()
        num_channels = wav_file.getnchannels()
        sample_rate = wav_file.getframerate()
        num_frames = wav_file.getnframes()

        # Read the audio data
        audio_data = wav_file.readframes(num_frames)

    # Convert the audio data to AudioSegment object
    audio = AudioSegment(
        data=audio_data,
        sample_width=sample_width,
        frame_rate=sample_rate,
        channels=num_channels
    )

    # Normalize the volume
    normalized_audio = audio.normalize()

    # Change the sampling rate to 1
    normalized_audio = normalized_audio.set_frame_rate(16000)

    # Combine all channels to one channel
    normalized_audio = normalized_audio.set_channels(1)

    # Export the normalized audio as WAV file
    normalized_audio.export(file_name, format='wav')


In [7]:
import pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture as GMM
from FeatureExtraction import extract_features
#from speakerfeatures import extract_features
import warnings
import os
import sys
import time
warnings.filterwarnings("ignore")

time1 = time.time()
training = True
dest = "models/"

training_size = 18

directory = output_folder

# 1 speaker
# directory = directories[0]

# List all files in the directory
file_paths = os.listdir(directory)
file_paths = sorted(file_paths)
# print(file_paths)
# Extracting features for each speaker
features = np.asarray(())
for path in file_paths:
    path = directory + "/" + path    
    path = path.strip()   
    print (path)
    
    # read the audio
    sr,audio = read(path)
    
    # extract 40 dimensional MFCC & delta MFCC features
    vector   = extract_features(audio,sr)
    
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
    # print(features.shape)
    # when features of 5 files of speaker are concatenated, then do model training
    # -> if count == 5: --> edited below

    # speaker_dir = mfcc_dir + directory
    # speaker_name = path.split("/")[1].split(".")[0]

    # if not os.path.exists(speaker_dir):
    #     os.makedirs(speaker_dir)
    # # Assuming 'features' is the 2D array containing the features
    # csv_path = speaker_dir + "/" + speaker_name + ".csv"
    # np.savetxt(csv_path, vector, delimiter=",")
    # print("Features saved to", csv_path)
    

if training:            
    gmm = GMM(n_components = 5, covariance_type='diag',n_init = 3)
    gmm.fit(features)
    # dumping the trained gaussian model
    picklefile = directory + ".gmm"
    print(picklefile)
    cPickle.dump(gmm,open(dest + picklefile,'wb'))
    print ('+ modeling completed for speaker:',picklefile," with data point = ",features.shape)   
    features = np.asarray(())
print("Total Time taken: ", round(time.time() - time1, 2), "seconds")


Silence_1/Silence_1_1.wav
Silence_1/Silence_1_10.wav
Silence_1/Silence_1_11.wav
Silence_1/Silence_1_12.wav
Silence_1/Silence_1_13.wav
Silence_1/Silence_1_14.wav
Silence_1/Silence_1_15.wav
Silence_1/Silence_1_16.wav
Silence_1/Silence_1_17.wav
Silence_1/Silence_1_2.wav
Silence_1/Silence_1_3.wav
Silence_1/Silence_1_4.wav
Silence_1/Silence_1_5.wav
Silence_1/Silence_1_6.wav
Silence_1/Silence_1_7.wav
Silence_1/Silence_1_8.wav
Silence_1/Silence_1_9.wav
Silence_1.gmm
+ modeling completed for speaker: Silence_1.gmm  with data point =  (3383, 22)
Total Time taken:  0.73 seconds
